In [3]:
import os
import base64
from dotenv import load_dotenv
from openai import OpenAI

In [4]:
load_dotenv()
client = OpenAI(api_key=os.getenv("API_KEY"))

In [5]:
def encode_image_to_base64(image_path: str) -> str:
    with open(image_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode("utf-8")


In [6]:
PROMPT = """
This is an image of a Japanese receipt.

Please extract the following information and return it in JSON format:

- Store name ("store")
- Date and time of purchase ("date")
- List of purchased items ("items"):
    Each item should include:
    - Name ("name")
    - Price in yen ("price")
- If there are discounts, include them as items with a negative price.
- Total amount ("total")
- Payment method and change ("payment.method" and "payment.change")

Return the result using this JSON structure:

{
  "store": "",
  "date": "",
  "items": [
    { "name": "", "price": 0 }
  ],
  "total": 0,
  "payment": {
    "method": "",
    "change": 0
  }
}
"""


In [7]:
def analyze_receipt_with_gpt4o(image_path: str):
    base64_image = encode_image_to_base64(image_path)

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": [
                    { "type": "text", "text": PROMPT },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        }
                    }
                ]
            }
        ],
        max_tokens=1000,
        temperature=0.2
    )

    print("📦 JSON Output:")
    print(response.choices[0].message.content)

In [8]:
if __name__ == "__main__":
    image_path = "./dataset/2.jpg"
    analyze_receipt_with_gpt4o(image_path)

📦 JSON Output:
```json
{
  "store": "MEGAドン・キホーテUNY浜松泉町店",
  "date": "2025年03月23日 18:18",
  "items": [
    { "name": "レジ袋", "price": 5 },
    { "name": "午後の紅茶 おいしい無糖", "price": 138 },
    { "name": "オイコス オーガニック", "price": 980 },
    { "name": "フライパン 直径25cm", "price": 128 },
    { "name": "朝日 ブラックペッパー", "price": 299 }
  ],
  "total": 2017,
  "payment": {
    "method": "現金",
    "change": 169
  }
}
```


In [2]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
client = OpenAI(api_key=os.getenv("API_KEY"))

# print(api_key := os.getenv("API_KEY"))

# response = client.models.list()
# for model in response.data:
#     print(model.id)


# API テスト